### Download data
Download data here https://www.kaggle.com/datasets/mikhailgaerlan/spongebob-squarepants-completed-transcripts

In [10]:
import os
import re

folder_path = "SpongeBob"

sponge_data = []
# Loop over all the files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    with open(file_path, "r") as file:
        document = []
        for line in file:
            clean_string = re.sub(r'\[.*?\]', '', line).strip()
            if clean_string:
                character, *utterance = clean_string.split(":")
                utterance = ':'.join(utterance).strip()
                character = character.strip()
                if utterance:
                    # Truncate up to 20 words
                    words = utterance.split()
                    trunc_utter = ' '.join(words[:20]) 
                    document.append({"utterance": trunc_utter, "character": character})
    sponge_data.append(document)

In [11]:
def merge_consecutive_utters_with_same_character(df):
    merged_data = []
    last_utter = None
    for doc in sponge_data:
        document = []
        for utterance in doc:
            if last_utter is not None and utterance['character'] == last_utter['character']:
                last_utter['utterance'] += ' ' + utterance['utterance']
            else:
                if last_utter is not None:
                    document.append(last_utter)
                last_utter = utterance.copy()
        if last_utter is not None:
            document.append(last_utter)
        merged_data.append(document)
    return merged_data

sponge_data = merge_consecutive_utters_with_same_character(sponge_data)


In [12]:
# Download personachat dataset
# !wget https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json

In [13]:
import json
import random

with open("personachat_self_original.json", 'r') as f:
    persona_data = json.load(f)


def get_random_sentences(n=5):
    sentences = []
    for i in range(n):
        random_persona = random.choice(persona_data['train'])
        random_utterance = random.choice(random_persona['utterances'])
        random_sentence = random.choice(random_utterance['candidates'])
        sentences.append(random_sentence)
    return sentences

In [14]:
dataset = {}

from personality import sponge_bob_personality

dataset['personality'] = sponge_bob_personality
dataset["utterances"] = []

SPONGE_BOB = 'SpongeBob'
last_utter = None
history = []
for doc in sponge_data:
    for utterance in doc:
        if last_utter is not None and last_utter['character'] != SPONGE_BOB and utterance['character'] != SPONGE_BOB:
            history = []

        if history and utterance['character'] == SPONGE_BOB:
            example = {}
            example['candidates'] = get_random_sentences()
            example['candidates'].append(utterance['utterance'])
            # Take no more than 5 utterances
            example['history'] = history.copy()[-5:]
            dataset['utterances'].append(example)

        history.append(utterance['utterance'])
        last_utter = utterance.copy()
    history = []
    last_utter = None

In [15]:
len(dataset["utterances"])

12023

### Split to train and test dataset for GPT chatbot

In [16]:
import json
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(dataset['utterances'], test_size=0.1)

train_dataset = {}
train_dataset['personality'] = sponge_bob_personality
train_dataset["utterances"] = train_data

with open('train.json', 'w') as f:
    json.dump([train_dataset], f)


test_dataset = {}
test_dataset['personality'] = sponge_bob_personality
test_dataset["utterances"] = test_data

with open('test.json', 'w') as f:
    json.dump([test_dataset], f)

### Create data file for simple chatbot

In [17]:
with open("sponge_bob_statements.txt", 'w') as f:
    for utterance in dataset["utterances"]:
        f.write(f"{utterance['candidates'][-1]}\n")

In [18]:

with open('sponge_bob_statements.txt', 'r') as f1:
    content1 = f1.read()

with open('../personachat_train_statements.txt', 'r') as f2:
    content2 = f2.read()

# Concatenate the contents of the two files
content = content1 + "\n" + content2

# Open the third file in write mode and write the concatenated contents to it
with open('personachat_sponge_bob_statements.txt', 'w') as f3:
    f3.write(content)